# Jupyter Notebook Example

this is how you use parallel I/O with a simple Jupyter widget with help from Hari Krishnan

In [1]:
5+ 5

10

In [4]:
%matplotlib inline
import imageio
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from ipywidgets import widgets, interact, Image
from ipyparallel import Client

from IPython.display import display, Javascript

plt.ion()

c = Client()
view = c[:]



w = HelloWidget(imageio.imread('frb_filters.jpg'))
display(w)
button = widgets.Button(description="Click Me!")
display(button)

@view.remote(block=True)
def mpi_rank():
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    return comm.Get_rank()

mpi_rank()
mpi_rank.block = False
ar = mpi_rank()
ar.get_dict()
#print(ar.get_dict())

#fig, ax = plt.subplots(1, 1)


def view_image(x_range, y_range):
    #print(x_range)
    #print(y_range)

    cmds = """
x_range = {0}
y_range = {1}


my_g = np.ix_(x_range,y_range)

result = im[np.ix_(x_range,y_range)]
return_value = comm.gather(result, root=0)
""".format(x_range, y_range)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.clf()
    #print(cmds)
    view.execute(cmds, block=True)
    image = view["im"][0]
    
    x_range = view["x_range"][0]
    y_range = view["y_range"][0]
    
    crop_image = view["return_value"][0][0]
   
    upper_left = (x_range[0], y_range[0])
    rect_length = (x_range[1] - x_range[0], y_range[1] - y_range[0])

    ax = plt.gca()

    #ax.imshow(image, cmap=plt.cm.gray_r)

    plt.title('View: ')
    rect = patches.Rectangle(upper_left, rect_length[0], rect_length[1],linewidth=1,edgecolor='r',facecolor='none')

    # Add the patch to the Axes
    #ax[0].add_patch(rect)
    ax.imshow(image, cmap=plt.cm.gray_r)
    ax.set_ylim(y_range[0], y_range[1])
    ax.set_xlim(x_range[0], x_range[1])
    #ax.imshow(crop_image, cmap=plt.cm.gray_r)
    plt.gca().invert_yaxis()
    fig.canvas.draw()
    #plt.show()

setup_code = """
from mpi4py import MPI
import h5py
import scipy.misc
import imageio
import numpy as np

filename = "~/Downloads/frb_filters.jpg"
im = imageio.imread(filename)
print(im.shape)
#im.shape = (im.shape[1], im.shape[0], im.shape[2])
print(im.shape)
im = im[:,:,0]


comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

"""
view.execute(setup_code, block=True)
def click(b):
    view_image((w._coords[0], w._coords[2]), (w._coords[1],w._coords[3]))
    #interactive(view_image, x_range=(w._coords[0], w._coords[2]), y_range= (w._coords[1],w._coords[3]))

button.on_click(click)


/anaconda/lib/python3.6/site-packages/ipyparallel/util.py:208: RuntimeWarning: IPython could not determine IPs for MayaNielanMacBookAir.dhcp.lbnl.us: [Errno 8] nodename nor servname provided, or not known
  RuntimeWarning)
/anaconda/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@MayaNielanMacBookAir.dhcp.lbnl.us')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


# Maya's widget

Backend 

In [1]:
import ipywidgets as widgets
from traitlets import Unicode, validate
import traitlets
import image_attendant as imat


class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    _events_active = traitlets.Bool(True, help='Indicate if canvas events are actively captured').tag(sync=True)
    _data_compressed = traitlets.Bytes(help='Compressed image data').tag(sync=True)
    _width = traitlets.CInt(help='Image display width (CSS pixels)').tag(sync=True)
    _AR = traitlets.Float(help='Image aspect ratio').tag(sync=True)
    _mime_type = traitlets.Unicode(help='Encoding format, e.g. PNG or JPEG').tag(sync=True)
    _event = traitlets.Dict(help='Canvas-generated event information').tag(sync=True)
    _coords = traitlets.List(help='rectangle coords').tag(sync=True)
    allow_pixelated = traitlets.Bool(True, help='Allow pixelated rendering when zoomed in').tag(sync=True)
    value = _coords
  
    def __init__(self, data=None, url=None, format='png', quality=70):
        """Instantiate a new Canvas Image Widget
        Image data must be compatible with a Numpy array with a shape similar to the following:
            (rows, columns)    - Greyscale
            (rows, columns, 1) - Greyscale
            (rows, columns, 3) - RGB
            (rows, columns, 4) - RGBA
        If data type is not np.uint8 it will be cast to uint8 by scaling
        min(data) -> 0 and max(data) -> 255.
        If you supply instead a URL to an image then it'll be fetched and displayed directly.
        When/if the user asks for it, the compressed image will be uncmpressed at the backend
        and made available as a numpy array via the .data attribute.
        The `quality` keyword is only used for 'webp' and 'jpg' image formats.
        """
        super().__init__()

        self._url = ''
        self._data = None
        self._format = ''

        self.format = format
        self.quality = quality

        if url:
            self.url = url
        else:
            self.data = data
            
        self._coords =[]

        # Manage user-defined Python callback functions for frontend events
        self._event_dispatchers = {}
    @property
    def coords(self):
        return self._coords
    
    @coords.setter
    def coords(self, value):
        self._coords = value
        
    @property
    def format(self):
        """Mime-type format, eg. png, jpg, webp
        """
        return self._format

    @format.setter
    def format(self, value):
        value = value.lower()
        valid_values = ['png', 'jpg', 'webp']
        if value not in valid_values:
            raise ValueError('Invalid image format: {}'.format(value))

        self._format = value
        self._mime_type = 'image/{}'.format(self._format)

    @property
    def url(self):
        return self._url

    @url.setter
    def url(self, value):

        self._url = value
        try:
            compressed, fmt = imat.download(self._url)
        except IOError:
            raise

        self._data = None
        self.format = fmt
        self._data_compressed = compressed

    @property
    def data(self):
        """Image data as Numpy array
        """
        if self._data is None and self._data_compressed:
            # This is for when widget is instantiated indirectly via image URL.
            # Only decompress the data at backend if user asks for it, like right now.
            self._data = imat.decompress(self._data_compressed)

        return self._data

    @data.setter
    def data(self, value):
        if value is None:
            # Reset to a small transparent rectangle
            self._data = None
            value = np.zeros((10, 15, 4), dtype=np.uint8)
        else:
            self._data = value

        with self.hold_sync():
            self.width = 0 #  self._data.shape[1]
            self.AR = 0 #  self._data.shape[0] / self._data.shape[1]
            self._data_compressed = imat.compress(value, self._format, quality=self.quality)

    @property
    def width(self):
        """Image display width in CSS pixels
        """
        return self._width

    @width.setter
    def width(self, value):
        if not value:
            value = self.data.shape[1]

        self._width = value

    @property
    def height(self):
        """Image display height in CSS pixels
        """
        return round(self._AR*self._width)

    @height.setter
    def height(self, value):
        self._width = value / self._AR

    @property
    def AR(self):
        """Image aspect ratio height / width
        """
        return self._AR

    @AR.setter
    def AR(self, value):
        if not value:
            value = self._data.shape[0] / self._data.shape[1]

        self._AR = value
        
    def sayhi(self):
        print("hellooo")

frontend

In [2]:
%%javascript
//TESTER
require.undef('hello');

function valid_size(value) {
    if (isNaN(value)) return false
    if (value == '')  return false

    return true
}

function notebook_cell_width(el) {
    // Determine width of notebook cell containing specified element.
    // http://stackoverflow.com/questions/22119673/find-the-closest-ancestor-element-that-has-a-specific-class
    // https://developer.mozilla.org/en-US/docs/Web/API/Element/closest
    var cls = '.output_subarea'
    var cell = el.closest(cls);


    if (cell == null) {
        return null
    } else {
        var padding = 6
        var width = cell.clientWidth - padding*2

        return width
    }
}

define('hello', ["@jupyter-widgets/base"], function(widgets) {
    var counter = 0;
    var mouse_down = [0,0]
    var image_save;
    var HelloView = widgets.DOMWidgetView.extend({
         handle_mouse_event: function(ev) {
        // General mouse-event handler
        if (this.model.get('_events_active')) {
            // Only deal with mouse events when flag is enabled
           // console.log("hey")
            // movementX, movementY
            var fields = ['shiftKey', 'altKey', 'ctrlKey', 'timeStamp', 'buttons', 'button']
            var pev = {'type': ev.type};
            for (let f of fields) {
                pev[f] = ev[f]
            }

            // Canvas-local XY coordinates
            // https://developer.mozilla.org/en-US/docs/Web/API/Element.getBoundingClientRect
            var rect = this.canvas.getBoundingClientRect();

            // Relative coordinates
            var X = (ev.clientX - rect.left) / (rect.right  - rect.left);
            var Y = (ev.clientY - rect.top)  / (rect.bottom - rect.top);

            // Data pixel coordinates
            pev.canvasX = Math.floor(X*this.canvas.width);
            pev.canvasY = Math.floor(Y*this.canvas.height);

            // Additional fields for `wheel` event
            // https://developer.mozilla.org/en-US/docs/Web/Reference/Events/wheel
            if (ev.type == 'wheel') {
                fields = ['deltaMode', 'deltaX', 'deltaY', 'deltaZ']
                for (let f of fields) {
                    pev[f] = ev[f]
                }
            }
           
            if (ev.type == 'mousedown') {
                console.log(image_save)
                this.ctx.drawImage(image_save, 0, 0);
                mouse_down = [pev.canvasX, pev.canvasY]
                 

                console.log(mouse_down.type)
            }
            if (ev.type == 'mouseup'){
                var coordx = pev.canvasX-mouse_down[0]
                var coordy = pev.canvasY-mouse_down[1]
                this.ctx.strokeRect(mouse_down[0], mouse_down[1], coordx, coordy)
                this.model.set('_coords', [mouse_down[0], mouse_down[1], pev.canvasX, pev.canvasY])
                console.log([mouse_down[0], mouse_down[1], pev.canvasX, pev.canvasY])
            }
            

            // Done
            ///console.log(pev)
            this.model.set('_event', pev);
            this.touch();
        }
}
        ,

        render: function() {
            this.canvas = document.createElement('canvas');
            this.setElement(this.canvas);
            //let view = this;
            
            // standard HTML DOM change from JS
            //let f = document.createElement("button");
           // let t = document.createTextNode("Click me");
        let title = document.createElement("h3");            
            //this.el.appendChild(title);
            //f.appendChild(t);
            //this.el.appendChild(f)
            ////////////////////////////////////////////////////////////////////////
             
            //this.el.append(this.canvas)
            var kind = '2d'  // 'webgl'   !!!!!
            this.ctx = this.canvas.getContext(kind);
            // initializing the form and the title values  
            //i.setAttribute('value', this.model.get('value'));
            title.textContent = this.model.get('value');
            
            // Listening to changes in the frontend input
            //f.addEventListener("input", (inputEvent => view.callback(inputEvent, f)), false);
            this.canvas.addEventListener("mouseup", this.handle_mouse_event.bind(this), false);
            this.canvas.addEventListener('mousedown',   this.handle_mouse_event.bind(this));
            this.canvas.addEventListener('wheel',       this.handle_mouse_event.bind(this));
            this.canvas.addEventListener('click',       this.handle_mouse_event.bind(this));
            this.canvas.addEventListener('contextmenu', this.handle_mouse_event.bind(this));
            this.canvas.addEventListener('dblclick',    this.handle_mouse_event.bind(this));
            this.canvas.addEventListener('mousemove',   this.handle_mouse_event.bind(this));

            this.canvas.onmousedown = function(ev) {
                ev.preventDefault();
            }
            this.canvas.onwheel = function(ev) {
                ev.preventDefault();
            }
            this.canvas.oncontextmenu = function(ev) {
                ev.preventDefault();
            }
            this.listenTo(this.model, 'change:_event', this.value_changed);
            this.listenTo(this.model, 'change:_data_compressed', this.update_data);
            this.listenTo(this.model, 'change:_width',           this.update_css);
            this.listenTo(this.model, 'change:_AR',              this.update_css);
            this.listenTo(this.model, 'change:allow_pixelated', this.update_css);

            // handle to access the DOM elements directly
            //this.input = i;  
            this.title = title;
            
            this.update();
            this.update_data();
        },
        
        update_data: function() {
        // https://developer.mozilla.org/en-US/docs/Web/API/Blob
        var options = {'type': this.model.get('_mime_type')}
        var blob = new Blob([this.model.get('_data_compressed')], options);

        var promise = createImageBitmap(blob);
        promise.then(this.draw.bind(this));
},
        draw: function(image) {
        // Draw image to the canvas
        this.canvas.width = image.width    // canvas size changes must be done prior to
        this.canvas.height = image.height  //  drawImage else the canvas is automatically erased
        image_save = image;
        console.log(image_save)
        this.update_css();
        console.log('hi')
        this.ctx.drawImage(image, 0, 0);
        //this.ctx.rect(0, 0, 50, 100);
        //this.ctx.stroke()
},
         check_pixelated: function() {
        // Image rendering quality via CSS style
        // auto: render pixelated if display size is greater than image size by a factor of 1.5 or more
        if (this.model.get('allow_pixelated')) {
            // pixelated rendering is allowed
            var thresh = 1.5
            if (this.model.get('_width') / this.canvas.width >= thresh) {
                return true
            } else {
                return false
            }
        } else {
            // pixelated rendering is disabled
            return false
        }
    },

    update_css: function() {
        // Update CSS display width and height.  No need to redraw canvas.
        // This function should be called prior to drawing so that any styles are in place
        // when the draw function is called.
        var width = this.model.get('_width');
        var AR = this.model.get('_AR');

        // Enforce notebook cell-width constraint
        var cell_width = notebook_cell_width(this.canvas);
        if (cell_width != null && width > cell_width) {
            width = cell_width
            this.model.set('_width', width);
            this.touch();
        }
        var height = Math.round(AR*width);

        // Check and set image-rendering quality
        if (this.check_pixelated()) {
            this.canvas.style.imageRendering = 'pixelated'
        } else {
            this.canvas.style.imageRendering = 'auto'
        }

        if (valid_size(width)) {
            this.canvas.style.width = width + 'px'
            this.canvas.style.min_width = width + 'px'
            this.canvas.style.max_width = width + 'px'
        } else {
            this.canvas.style.width = null       // do I really need these nulls here??
            this.canvas.style.min_width = null
            this.canvas.style.max_width = null
        }
        if (valid_size(height)) {
            this.canvas.style.height = height + 'px'
            this.canvas.style.min_height = height + 'px'
            this.canvas.style.max_height = height + 'px'
        } else {
            this.canvas.style.height = null       // do I really need these nulls here??
            this.canvas.style.min_height = null
            this.canvas.style.max_height = null
        }
},


        
        value_changed: function() {
            // access to the 'input' DOM element
            //this.input.setAttribute('value', this.model.get('value'))
            // access to the 'h3' DOM element
           // console.log('value_change')
           this.title.textContent = this.model.get("_event")

        }
    });
    
    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

In [3]:
x =(imageio.imread('frb_filters.jpg'))

NameError: name 'imageio' is not defined

In [14]:
w2 = HelloWidget(x)